In [1]:
import sys
import os

# Add the parent directory to the path so the package is importable
sys.path.append(os.path.abspath(".."))

from llm_data_quality_assistant.enums import Models
import pandas as pd
from pprint import pprint
from dotenv import load_dotenv
import numpy as np
import jupyter_helper_functions
import string
import time

load_dotenv()

True

# 2. Load and Explore EudraCT Data
Load the EudraCT dataset and perform exploratory data analysis to understand its structure and content.

In [2]:
corrupt_dataset = jupyter_helper_functions.load_dataset(
    "../datasets/parker_datasets/eudract/parker-repair/eudract_repair.csv"
)
gold_standard = jupyter_helper_functions.load_dataset(
    "../datasets/parker_datasets/eudract/eudract_cleaned_gold_first1000.csv"
)

partial_keys = jupyter_helper_functions.load_text_file(
    "../datasets/parker_datasets/eudract/eudract.partialkey"
)
rules = jupyter_helper_functions.load_text_file(
    "../datasets/parker_datasets/eudract/eudract.rules"
)

# print(partial_keys)
# print(rules)
# print(corrupt_dataset.head(2))
# print(gold_standard.head(2))
# print(type(gold_standard.get("eudract_number").iloc[0]))
# print(type(corrupt_dataset.get("eudract_number").iloc[0]))

# 3. Clean and Merge Data with LLM
Use the LLM pipeline to clean and merge the corrupted dataset using the provided rules and evaluate the results.

In [ ]:
from llm_data_quality_assistant.pipeline import Pipeline
from llm_data_quality_assistant.enums import Models
import string
import json
import jupyter_helper_functions

# Use a primary key for merging
primary_key = "eudract_number"
model = Models.GeminiModels.GEMINI_2_0_FLASH_LITE
# model = Models.OpenAIModels.GPT_4_1_MINI
rows_of_context = 50

extra = "parker_postprocessing_option_2"
file_name = jupyter_helper_functions.sanitize_filename(f"{model.value}_{rows_of_context}_rows_context_{extra}")   

rpm = 25
additional_prompt = f"""
Here are rows of the dataset to provide context for the cleaning process:
{corrupt_dataset.sample(rows_of_context).to_string(index=False)}
"""

# Merge/clean with LLM
merged_df, time_taken = jupyter_helper_functions.merge_with_llm_timed(
    dataset=corrupt_dataset,
    primary_key=primary_key,
    model=model,
    rpm=rpm,
    additional_prompt=additional_prompt
)
# time_taken = 0


In [ ]:
# Save merged dataset
# jupyter_helper_functions.save_dataframe_csv(merged_df, f"../analysis/repairs/eudract/merged_dataset_{file_name}.csv")

original_corrupted_datasets = jupyter_helper_functions.load_dataset(
    "../datasets/parker_datasets/eudract/eudract_corrupted_first1000.csv"
)

# merged_df = jupyter_helper_functions.load_dataset(
#     "../analysis/repairs/eudract/merged_dataset_gemini_2_0_flash_lite_50_rows_context_o2_combined.csv"
# )
# Evaluate results
jupyter_helper_functions.standardize_and_evaluate(
    gold_standard=gold_standard,
    merged_df=merged_df,
    corrupt_dataset=original_corrupted_datasets,
    primary_key=primary_key,
    time_delta=time_taken,
    results_dir=f"../analysis/results/eudract/",
    file_name=file_name,
)

{'accuracy': 0.9429017271340923,
 'column_names': ['eudract_number',
                  'arms',
                  'controlled',
                  'crossover',
                  'double_blind',
                  'open',
                  'parallel_group',
                  'randomised',
                  'single_blind'],
 'f1_score': 0.6521175453759723,
 'false_negative': 1453,
 'false_negative_rate': 0.49054692775151926,
 'false_positive': 157,
 'false_positive_rate': 0.006221517733306915,
 'num_columns': 9,
 'num_rows': 3133,
 'precision': 0.9057623049219687,
 'recall': 0.5094530722484808,
 'time_taken': 0,
 'true_negative': 25078,
 'true_positive': 1509}
{'column_names': ['eudract_number',
                  'arms',
                  'controlled',
                  'crossover',
                  'double_blind',
                  'open',
                  'parallel_group',
                  'randomised',
                  'single_blind'],
 'num_columns': 9,
 'num_rows': 3133,
 'stats': 